In [1]:
import lightkurve as lk
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from astropy.coordinates import SkyCoord

from multiprocessing import Pool

from tqdm import tqdm
import os
import glob

from Kakapo.build_epsf import epsf_data_creation
from Kakapo.kakapo import Kakapo

import warnings

warnings.filterwarnings('ignore')

%matplotlib widget

In [2]:
def uno_reverse():

    # List of directories to clean
    directories = [
        "filtered_stars",
        "csv_files",
        "detected_events",
        "difference_arrays",
        "object_ids", 
        "figures"
    ]
    
    for directory in directories: # Iterate over each directory
        folder_path = os.path.join(os.getcwd(), directory) # Create the full path to the directory
        
        if os.path.exists(folder_path) and os.path.isdir(folder_path): # Check if the folder exists
            files = glob.glob(os.path.join(folder_path, "*")) # List all files in the directory using glob
            
            for file in files: # Iterate through and remove each file
                try:
                    os.remove(file)
                    # print(f"Deleted: {file}")
                except Exception as e:
                    print(f"Failed to delete {file}: {e}")
        else:
            print(f"Directory does not exist: {folder_path}")


In [3]:
uno_reverse()

In [4]:

plt.rc('text', usetex=True)
plt.rc('font', family='serif')

fig_width_pt = 244.0  # Get this from LaTeX using \the\columnwidth
text_width_pt = 508.0 # Get this from LaTeX using \the\textwidth

inches_per_pt = 1.0/72.27               # Convert pt to inches
golden_mean = (np.sqrt(5)-1.0)/2.0         # Aesthetic ratio
fig_width = fig_width_pt*inches_per_pt*1.5 # width in inches
fig_width_full = text_width_pt*inches_per_pt*1.5  # 17
fig_height =fig_width*golden_mean # height in inches
fig_size = [fig_width,fig_height] #(9,5.5) #(9, 4.5)
fig_height_full = fig_width_full*golden_mean

In [ ]:
ra = 202.9625
dec = -10.736

c = SkyCoord(ra=ra,dec=dec,unit='deg')
res = lk.search_targetpixelfile(c, mission='K2', campaign = 6, radius = 1200)#[0].download()

tpfs_res = res.download_all(quality_bitmask=0)

# tpfs_res[0].targetid

In [ ]:

epsf_data = epsf_data_creation(tpfs_res)

# print(epsf_data[2:-2,2:-2])

plt.figure()
plt.imshow(epsf_data[2:-2,2:-2], origin = 'lower')
plt.show()

In [ ]:
m = 25

kea = Kakapo(tpfs_res[:m], epsf_data[2:-2, 2:-2], plot=False, num_cores=None, 
             mask_value=1000, tol=0.003, std1 = 3.0, std2 = 3.0, 
             detect = True, diagnostic_plots = False, save = True, filtered = True,
             overwrite = True, corrlim=0.2, difflim=0.6, fwhmlim=2.5, maxlim=0, snrlim=5,
             roundness=0.8, perc_val_limit=0.5, poiss_val=3, savepath='figures')

In [8]:
# Kepler and K2 Analysis of Photometric Objects


